# 综述

任何交易算法的最终目标是在每个时间点保持“最佳”可能的投资组合，但算法在“最佳”的定义中有所不同。一种算法可能希望基于对未来价格的预测来最大化预期回报的投资组合。另一种算法可能需要在固定的多头和空头组合中，尽可能保持接近的权重。

许多算法还希望确保“最佳”组合满足一些约束条件。想要使预期回报最大化的算法可能希望对其投资组合的总市值设置上限，希望保持等权重多空投资组合的算法可能希望限制其每日换手率。

寻找受限制的“最佳”投资组合的一个强大技术是以投资组合优化问题的形式来构建任务。

投资组合优化问题是以下形式的数学问题：
>给定一个目标函数F和一个不等式约束列表$C_i≤h_i$，找到一个向量w的投资组合权重，使F最大化，同时满足每个约束。

数学符号：

![数学符号](notation.png)

举例

一种算法建立一个模型，预测股票列表的预期收益。 该算法希望为这些股票分配有限数量的资金，使其获得最大可能的预期回报，而不会对任何单一股票下注太大。

我们可以将算法的目标表达为一个数学优化问题，如下所示：

令$\boldsymbol{r}=[r_0,r_1,...r_n]$为`n`个股票预期收益矢量。设$w_{max}$为任何单个股票允许的最大权重，并且$W_{max}$为整个投资组合的最大总权重。找到投资组合权重矢量，$$\boldsymbol{w}=[w_0,w_1,...w_n]$$求解问题:

![求解问题](problem.png)

像`CVXOPT`和`scipy.optimize`这样的`Python`库可以解决许多种优化问题，但它们要求用户以复杂的标准形式表达问题，这往往需要深入理解底层解决方法。像`cvxpy`这样的库建模提供了一个更高层次的接口，允许用户用抽象的数学表达式来描述问题，但是这仍然要求用户在金融领域概念和抽象数学概念之间来回翻译，这可能是单调乏味和容易出错的。

`optimize`模块提供了工具，可以直接根据财务领域概念定义和求解投资组合优化问题。用户向运行组合优化的API函数提供`Objective`和一系列约束对象，来与`Optimize API`交互。`Optimize API`隐藏了组合优化的大部分复杂数学部分，允许用户根据“最大化预期收益”和“限制行业风险”等高级概念来思考，而不是抽象矩阵相乘。

使用`Optimize API`，我们将解决上面的示例优化，如下所示：

# 运行优化

有三种方式可以使用优化API运行优化：
+ `zipline.optimize.calculate_optimal_portfolio()` 在满足约束列表的同时计算优化目标的投资组合。
+ `zipline.algorithm.order_optimal_portfolio()` 运行与`calculate_optimal_portfolio()`相同的优化，并放置实现该组合所必需的订单。
+ `zipline.optimize.run_optimization()`执行与`calculate_optimal_portfolio()`相同的优化，但会返回带有附加信息的`OptimizationResult`

`calculate_optimal_portfolio()`是笔记本研究环境中`Optimize API`的主要接口。

`order_optimal_portfolio()`是交易算法中Optimize API的主要接口。

`run_optimization()`是一个较低级别的API。它主要用于调试失败或产生意外结果的优化。

# `目标(Objectives)`

每一个投资组合优化都需要一个`Objective`来告诉优化器新投资组合该使用哪个函数来最大化。

目前有两个`Objective`：
+ `zipline.optimize.MaximizeAlpha`
+ `zipline.optimize.TargetWeights`

`MaximizeAlpha`用于算法预测期望收益率。`MaximizeAlpha`使用`Series`作为参数，该参数为每个资产的“alpha”值，`MaximizeAlpha`试图找到一组新的投资组合权重，使每个资产的权重乘以其alpha值的总和最大。

`TargetWeights`由显式构建目标投资组合的算法使用。例如，在识别目标资产列表的算法（例如，50个多头和50个空头列表）中，仅仅想要针对这些资产构造等重或等风险组合，这是有用的。`TargetWeights`使用一个`Series`参数，来映射资产目标权重。任务是找到(通过欧几里得距离)一组投资组合权重尽可能接近目标值。

# 限制条件(`Constraints`)

通常，投资组合优化器执行优化时是带限制条件的。例如，当使用`MaximizeAlpha`时，需要限定多头和空头头寸的总价值约束。如果我们不提供这样的约束，优化器将无法将“无限”资本分配到非零阿尔法值的每项资产中。

在运行优化时，传递约束对象列表给优化器，限定投资组合约束。例如，要求优化器产生一个总风险小于或等于当前投资组合值的投资组合，我们提供了一个`MaxGrossExposure`约束：

最常用的约束条件如下：
+ `MaxGrossExposure`将投资组合的总风险（即投资组合头寸绝对值之和）限制为小于当前投资组合价值的百分比
+ `NetExposure`将投资组合的净风险敞口（即投资组合的多头价值减去其空头价值）限制在当前投资组合价值的两个百分比之间
+ `PositionConcentration`约束了投资组合中每个单独资产的风险敞口
+ `NetGroupExposure`对某一市场分组的投资组合限制净风险敞口
+ `FactorExposure`对一组风险因子的投资组合限制净加权风险敞口

查看API文档的[约束部分](https://www.quantopian.com/help#constraints-api-docs)以及所有可用约束的完整列表。

# 调试优化

用户在使用Optimize API时可能遇到的一个问题是，可能会意外地要求优化器求解无解的问题。有两种常见的方式可能发生：

1. 没有满足所有必需的约束条件的投资组合。发生这种情况时，优化器会引发一个`InfeasibleConstraints`异常
2. 提供给优化器的约束无法使目标函数最大化上限。发生这种情况时，优化器会引发`UnboundedObjective`异常

调试`InfeasibleConstraints`约束可能更具挑战性。如果优化器触发`InfeasibleConstraints`异常时，则意味着**每一组可能的投资组合权重都至少违反了其中一条约束条件**。由于不同的投资组合可能会违反不同的约束条件，因此可能不会有单一的约束条件是罪魁祸首。

在调试`InfeasibleConstraints`异常时，我们可以利用一个优势，那就是我们可能期望某些特殊的投资组合可行。 例如，应该始终能够平仓现有的头寸以生成“空的”投资组合。当引发`InfeasibleConstraints`错误时，优化器会检查少数这些特殊投资组合，并生成错误消息，详细说明每个特殊投资组合违反的约束条件。用于诊断时，优化器目前可检查的投资组合是：
1. 当前投资组合
2. 空投资组合
3. 目标投资组合(仅在使用`TargetWeights`时适用)

例子

假设我们有一个投资组合，目前AAPL和MSFT中有多头和空头头寸，分别相当于投资组合价值的10％。以下优化将失败并出现`InfeasibleConstraints`错误：

产生的错误将包含以下消息：

此错误消息告诉我们，优化器测试的每个投资组合违反了不同的约束条件。目标投资组合违反了`DollarNeutral`约束，因为其净投资组合风险敞口等于投资组合总价值的10％。当前投资组合违反了`LongOnly`约束，因为它在MSFT中有空头头寸。空投资组合违反了AAPL的`FixedWeight`约束，因为它给AAPL权重为0.0。